# Using Selenium, Beatutiful Soup & Requests: _Hershel, O My Bag & Whimsy and Row_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Hershel, O My Bag & Whimsy and Row. We will get the: picture,price, product link, and description.

Resources used to help in this process can be found in other notebooks.

#### Import your needed Libraries:

In [50]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
from PIL import Image
import requests
from io import BytesIO
from scraping_functions import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on _Hershel, O My Bag & Whimsy and Row_.

In [2]:
hershel_backpacks = 'https://herschel.com/shop/womens/backpacks'
hershel_fp = 'https://herschel.com/shop/womens/hip-packs'
hershel_totes = 'https://herschel.com/shop/womens/totes'
hershel_messenger = 'https://herschel.com/shop/womens/messengers'

O_my_bag = 'https://www.omybag.nl/product-category/women/women-bags/'

#only the first 13-ish bags
WR_bags = 'https://whimsyandrow.com/collections/bags-clutches-pouches?sort_by=title-ascending'

### O My Bag:

Now that we have our page variables set up, we can scrape the bags. After inspecting on the website it appears we can just use beautiful soup for this task.

In [5]:
omb_page = requests.get(O_my_bag)
omb_soup = BeautifulSoup(omb_page.content, 'lxml')
divs_imgs = omb_soup.findAll('div', {'class':'content-product'})

In [13]:
omb_links = omb_soup.findAll('a', {'class':'woocommerce-LoopProduct-link'})
omb_href = [x['href'] for x in omb_links]

Now let's grab the images:

In [25]:
omb_images_tags = omb_soup.findAll('img', {'class':'attachment-woocommerce_thumbnail'})
omb_images = [x['src'] for x in omb_images_tags]

Now, let's get into the details like price and name:

In [27]:
omb_prod_details = omb_soup.findAll('div', {'class':'product-details'})

In [38]:
omb_names = [x.find('div', {'class': 'product-title'}).text.strip() for x in omb_prod_details]
omb_prices = [x.find('span', {'class': 'price'}).text.strip() for x in omb_prod_details]

### Create a DataFrame and use our function to download pictures :

Below we will use the lists created above and make a dataframe. Then we will define a function to download the images into our directory, and save the image file name in a new column of the DataFrame.

In [57]:
table = [omb_names,omb_prices,omb_href,omb_images]
OMB_df = pd.DataFrame(table).transpose()

In [58]:
OMB_df.rename( columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

In [59]:
OMB_df.head(3)

,product_name,product_price,URL,img_url
0,Lola – Black Croco – Soft Grain / Classic Leather,€169.00,https://www.omybag.nl/shop/women/lola-black-so...,https://static1.omybag.nl/wp-content/uploads/2...
1,Jean Backpack – Black – Soft Grain Leather,€299.00,https://www.omybag.nl/shop/women/jean-backpack...,https://www.omybag.nl/wp-content/uploads/2019/...
2,Mila (short handle) – Natural – Classic Leather,€329.00,https://www.omybag.nl/shop/men/mila-short-hand...,https://static2.omybag.nl/wp-content/uploads/2...


#### Save all images using an apply

In [65]:
c = 0
OMB_df['img'] = OMB_df.img_url.apply(lambda x: save_src_image_apply('../../data/','omybag_',x))

Saved omybag__1.png
Saved omybag__2.png
Saved omybag__3.png
Saved omybag__4.png
Saved omybag__5.png
Saved omybag__6.png
Saved omybag__7.png
Saved omybag__8.png
Saved omybag__9.png
Saved omybag__10.png
Saved omybag__11.png
Saved omybag__12.png
Saved omybag__13.png
Saved omybag__14.png
Saved omybag__15.png
Saved omybag__16.png
Saved omybag__17.png
Saved omybag__18.png
Saved omybag__19.png
Saved omybag__20.png
Saved omybag__21.png
Saved omybag__22.png
Saved omybag__23.png
Saved omybag__24.png
Saved omybag__25.png
Saved omybag__26.png
Saved omybag__27.png
Saved omybag__28.png
Saved omybag__29.png
Saved omybag__30.png
Saved omybag__31.png
Saved omybag__32.png
Saved omybag__33.png
Saved omybag__34.png
Saved omybag__35.png
Saved omybag__36.png
Saved omybag__37.png
Saved omybag__38.png
Saved omybag__39.png
Saved omybag__40.png
Saved omybag__41.png
Saved omybag__42.png
Saved omybag__43.png
Saved omybag__44.png
Saved omybag__45.png
Saved omybag__46.png
Saved omybag__47.png
Saved omybag__48.png
S

In [67]:
OMB_df.to_csv('../../omb_df.csv')

### Whimsy and Row:

In [73]:
wr_page = requests.get(WR_bags)
wr_soup = BeautifulSoup(wr_page.content, 'lxml')
divs_wr = wr_soup.findAll('div', {'class':'product-frame'})

In [106]:
wr_url = ['https://whimsyandrow.com' + x.find('a')['href'] for x in divs_wr[:-3]]
wr_names = [x.find('a')['title'] for x in divs_wr[:-3]]
wr_images_url = ['https:'+x.find('img')['src'] for x in divs_wr[:-3]] 

Now, let's get the prices which are in a different div:

In [97]:
divs_wr_2 = wr_soup.findAll('div', {'class':'product-title'})
prices = [x.find('h6').text for x in divs_wr_2[:-3]]

### Create this whimsy and Row DF and save the images:

In [107]:
table = [wr_names,prices,wr_url,wr_images_url]
wr_df = pd.DataFrame(table).transpose()
wr_df.rename(columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

#### Save all images using an apply:

In [112]:
c = 0
wr_df['img'] = wr_df.img_url.apply(lambda x: save_src_image_apply('../../data/','whimsy_row_',x))

Saved whimsy_row__1.png
Saved whimsy_row__2.png
Saved whimsy_row__3.png
Saved whimsy_row__4.png
Saved whimsy_row__5.png
Saved whimsy_row__6.png
Saved whimsy_row__7.png
Saved whimsy_row__8.png
Saved whimsy_row__9.png
Saved whimsy_row__10.png
Saved whimsy_row__11.png
Saved whimsy_row__12.png
Saved whimsy_row__13.png


In [113]:
wr_df.to_csv('../../whimsy_row_df.csv')

### _Hershel_:

In [115]:
h_page_1 = requests.get(hershel_backpacks)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [235]:
#three of the bookbags do not have multiple colors
colors_carousel_tester = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]

images_h_bb = [i.findAll('img',{'class':'swatch'}) if i.findAll('img',{'class':'swatch'}) 
               else i.find('img',{'class':'product__image'}) for x in colors_carousel for i in x]


#the script grabbed each bag twice, so this fixes that
images_new_bb = images_h_bb[::2]

In [206]:
h_bookbag_colors = [[i['data-color'] for i in x] for x in images_new_bb]
h_bookbag_img = [['https://herschel.com'+i['data-product-image'] for i in x] for x in images_new_bb]

In [212]:
names = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]

In [215]:
names_adj = []
for i in range(len(names)):
    n = len(h_bookbag_img[i])
    names_adj.extend([names[i]]*n)

IndexError: list index out of range

In [216]:
len(h_bookbag_img),len(names)

(94, 98)

In [217]:
len(divs_h)

98

In [218]:
len(images_new_bb)

94

In [220]:
colors_carousel

[[<div class="carousel js-slick-swatches">
  <div class="slide">
  <img class="swatch active" data-color="Black" data-product-image="/content/dam/herschel/products/10501/10501-00001-OS_01.jpg" data-sku="10501-00001-OS" src="/content/dam/herschel/swatches/1063.png"/>
  </div>
  <div class="slide">
  <img class="swatch " data-color="Ash Rose" data-product-image="/content/dam/herschel/products/10501/10501-02077-OS_01.jpg" data-sku="10501-02077-OS" src="/content/dam/herschel/swatches/02077.png"/>
  </div>
  <div class="slide">
  <img class="swatch " data-color="Woodland Camo" data-product-image="/content/dam/herschel/products/10501/10501-00032-OS_01.jpg" data-sku="10501-00032-OS" src="/content/dam/herschel/swatches/00032.png"/>
  </div>
  <div class="slide">
  <img class="swatch " data-color="Black Crosshatch" data-product-image="/content/dam/herschel/products/10501/10501-02090-OS_01.jpg" data-sku="10501-02090-OS" src="/content/dam/herschel/swatches/02090.png"/>
  </div>
  <div class="slid

In [221]:
colors_carousel_tester = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
# images_h_bb = [i.findAll('img',{'class':'swatch'}) for x in colors_carousel for i in x]

In [222]:
len(colors_carousel_tester)

98

In [223]:
len(colors_carousel)

98

In [227]:
colors_carousel_tester[18]

<div class="product-image">
<a class="js-product-grid-link" data-href="/shop/backpacks/settlement-backpack-studio" data-use-exact="false" href="/shop/backpacks/settlement-backpack-studio?v=10005-02161-OS">
<div class="ratio--4x5">
<img alt="Settlement Backpack | Studio" class="product__image lazyload" data-src="/content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.680.850.jpg" data-srcset="/content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.680.850.jpg 680w, /content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.300.375.jpg 330w, /content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.1000.1250.jpg 1000w, /content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.1000.1250.jpg 2500w" sizes="(min-width: 992px) 33.33vw, (max-width: 991px) and (min-width: 768px) 50vw, 100vw"/>
</div>
</a>
</div>

In [244]:
len(images_h_bb)

188

In [253]:
test = [i.findAll('img',{'class':'swatch'}) if i.findAll('img',{'class':'swatch'}) 
               else i.find('img') for x in colors_carousel_tester for i in x]

AttributeError: 'NavigableString' object has no attribute 'findAll'

In [252]:
colors_carousel_tester[18].find('img')

<img alt="Settlement Backpack | Studio" class="product__image lazyload" data-src="/content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.680.850.jpg" data-srcset="/content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.680.850.jpg 680w, /content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.300.375.jpg 330w, /content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.1000.1250.jpg 1000w, /content/dam/herschel/products/10005/10005-02161-OS_01.jpg.sthumbnails.1000.1250.jpg 2500w" sizes="(min-width: 992px) 33.33vw, (max-width: 991px) and (min-width: 768px) 50vw, 100vw"/>

In [249]:
for i in colors_carousel_tester:
    print(type(i))

<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.ResultSet'>
<class 'bs4.el